### 6. Связанные модели и запросы с отношениями

Django ORM поддерживает запросы к связанным моделям, что позволяет работать с отношениями между моделями, такими как **ForeignKey**, **ManyToMany**, и **OneToOne**. Эти отношения позволяют создавать запросы, извлекать данные из связанных таблиц, а также оптимизировать запросы с помощью методов **`select_related()`** и **`prefetch_related()`**.

#### 1. Запросы с ForeignKey и ManyToMany

Когда модели в Django связаны через **ForeignKey** или **ManyToMany**, можно делать запросы, которые включают данные из связанных моделей.

##### ForeignKey

**ForeignKey** создаёт связь "один ко многим", то есть одна запись одной модели может быть связана с несколькими записями другой модели.

Пример:

In [ ]:
class Author(models.Model):
    name = models.CharField(max_length=100)

class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.ForeignKey(Author, on_delete=models.CASCADE)

В данном случае книга (**Book**) связана с автором (**Author**) через **ForeignKey**.

Чтобы получить автора книги:

In [ ]:
# Получить книгу
book = Book.objects.get(id=1)
# Получить автора этой книги
author = book.author
print(author.name)

Чтобы найти все книги, написанные определённым автором:

In [ ]:
# Получить автора
author = Author.objects.get(id=1)
# Найти все книги, которые написал этот автор
books = Book.objects.filter(author=author)

##### ManyToMany

**ManyToMany** создаёт связь "многие ко многим", что означает, что одна запись одной модели может быть связана с множеством записей другой модели, и наоборот.

Пример:

In [ ]:
class Student(models.Model):
    name = models.CharField(max_length=100)

class Course(models.Model):
    title = models.CharField(max_length=100)
    students = models.ManyToManyField(Student)

Для того чтобы добавить студента в курс:

In [ ]:
# Получить студента и курс
student = Student.objects.get(id=1)
course = Course.objects.get(id=1)
# Добавить студента в курс
course.students.add(student)

Чтобы получить всех студентов, которые учатся на курсе:

In [ ]:
# Получить всех студентов курса
students = course.students.all()

Аналогично, чтобы получить все курсы, на которых учится студент:

In [ ]:
# Получить все курсы студента
courses = student.course_set.all()

#### 2. Метод `select_related()` и `prefetch_related()` для оптимизации запросов

Когда работаешь с связанными моделями, Django автоматически делает дополнительные запросы к базе данных для извлечения данных из связанных моделей. Это может привести к множественным запросам (так называемая проблема **"N+1 запросов"**). Чтобы избежать этого и оптимизировать работу с базой данных, Django предлагает методы **`select_related()`** и **`prefetch_related()`**.

##### `select_related()`

**`select_related()`** используется для оптимизации запросов, которые включают в себя **ForeignKey** и **OneToOne** отношения. Этот метод делает **JOIN** запрос на уровне базы данных, загружая данные связанных моделей в одном запросе.

Пример:

In [ ]:
# Получить все книги вместе с авторами с помощью select_related()
books = Book.objects.select_related('author').all()

for book in books:
    print(book.title, book.author.name)

Вместо того чтобы делать отдельный запрос для каждой книги, Django сделает один запрос с JOIN для получения как книги, так и её автора.

**Когда использовать `select_related()`**:
- Когда нужно получить объекты, связанные через **ForeignKey** или **OneToOne**, и данные связанной модели точно будут использованы.
- Этот метод удобен, если есть отношение "один к одному" или "один ко многим".


##### `prefetch_related()`

**`prefetch_related()`** используется для оптимизации запросов с отношениями **ManyToMany** и **ForeignKey**, но в отличие от **`select_related()`**, он делает несколько запросов, а затем "связывает" их в памяти, минимизируя количество запросов к базе данных.

Пример:

In [ ]:
# Получить всех студентов и их курсы с помощью prefetch_related()
students = Student.objects.prefetch_related('course_set').all()

for student in students:
    print(student.name)
    for course in student.course_set.all():
        print(course.title)

В данном случае **`prefetch_related()`** делает два отдельных запроса: один для получения всех студентов, а другой для получения всех связанных курсов. Но он избегает множества отдельных запросов для каждого студента, связывая данные в памяти.

**Когда использовать `prefetch_related()`**:
- Когда нужно загрузить связанные объекты через **ManyToMany** или **ForeignKey** (в обратную сторону).
- Этот метод полезен для отношений "многие ко многим" и когда есть необходимость сделать несколько отдельных запросов и связать их на уровне Python.

##### Пример сравнения `select_related()` и `prefetch_related()`

Представим, что у нас есть авторы и книги, и мы хотим получить всех авторов и их книги:

1. **Использование `select_related()`**:

In [ ]:
# Используем select_related для ForeignKey связи
books = Book.objects.select_related('author').all()

for book in books:
    print(book.title, book.author.name)

   Этот запрос будет эффективен для связи **ForeignKey**, так как он выполнит **JOIN** и получит все необходимые данные за один запрос.

2. **Использование `prefetch_related()`**:

In [ ]:
# Используем prefetch_related для ManyToMany связи
courses = Course.objects.prefetch_related('students').all()

for course in courses:
    print(course.title)
    for student in course.students.all():
        print(student.name)

Этот метод выполнит два запроса: один для получения всех курсов, и один для получения всех студентов, но это лучше, чем множество запросов для каждого курса.

Использование методов **`select_related()`** и **`prefetch_related()`** помогает избежать лишних запросов и значительно улучшить производительность приложения. Студенты научатся правильно оптимизировать работу с базой данных в Django ORM, особенно при работе со сложными и связанными моделями.